In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import gitsummary
print(gitsummary.print_cwd_git_version())
import socket; print(f"Running on: {socket.getfqdn()}")

No git repository detected.
Running on: login08.chn


# MERRA2 - Guan Waliser v2

This section uses Guan and Waliser to determine AR presence over Bloomington

In [2]:
""" Open the GW ARTMIP files """

algorithm = "guan_waliser"
artmip_base =  "/global/cfs/cdirs/m1517/cascade/external_datasets/ARTMIP_NCAR_mirror/catalogues/tier1/"
file_glob = f"{artmip_base}/{algorithm}/MERRA2.ar_tag*.nc4"

import glob
# find all the netcdf files in the directory
files = sorted(glob.glob(file_glob))

# for GW only, remove the file that contains all the data
files = [ f for f in files if "1980to2017" not in f ]

gw_xr = xr.open_mfdataset(files)
gw_xr

<xarray.Dataset>
Dimensions:        (time: 109568, lat: 361, lon: 576, ens: 1, lev: 1,
                    limitID: 5, stageID: 9)
Coordinates:
  * ens            (ens) float64 1.0
  * lat            (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lev            (lev) float64 1e+03
  * limitID        (limitID) float64 1.0 2.0 3.0 4.0 5.0
  * lon            (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * stageID        (stageID) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0
  * time           (time) datetime64[ns] 1980-01-01 ... 2017-06-30T21:00:00
Data variables: (12/32)
    ar_binary_tag  (time, lat, lon) int8 dask.array<chunksize=(2928, 361, 576), meta=np.ndarray>
    axis           (ens, time, lev, lat, lon) float64 dask.array<chunksize=(1, 2928, 1, 361, 576), meta=np.ndarray>
    clat           (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    clon           (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    day            (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    hlat           (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    ...             ...
    tlon           (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    tnsct          (ens, time, lev, lat, lon) float64 dask.array<chunksize=(1, 2928, 1, 361, 576), meta=np.ndarray>
    width          (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    width2         (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
    xtnsctidx      (ens, time, lev, lat, lon) float32 dask.array<chunksize=(1, 2928, 1, 361, 576), meta=np.ndarray>
    year           (ens, time, lev, lat) float32 dask.array<chunksize=(1, 2928, 1, 361), meta=np.ndarray>
Attributes:
    title:           Global Atmospheric River Database
    version:         2017.11.05
    creation_date:   05-Nov-2017 18:50:33
    tARget_version:  1.229_ARTMIP
    reference:       Guan, B., and D. E. Waliser (2015), Detection of atmosph...
    history:         Fri Feb  8 17:51:38 2019: ncks -d time,0,2927 /ftp/archi...
    NCO:             "4.5.2"

In [3]:
# verify that all time values are unique
assert len(gw_xr.time) == len(np.unique(gw_xr.time))

In [4]:
""" Pull out a point over Bloomington """
blat, blon = 39.1653, -86.5264 

# select the given lat/lon
gw_bloomington = gw_xr.sel(lat = blat, lon = blon, method = 'nearest').squeeze()
# create a time index variable, used later on in the composite process
gw_bloomington['indices'] = xr.DataArray(np.arange(0,len(gw_bloomington.time), dtype = int), dims = ('time',))
# pull out only the ar_binary_tag variable and indices variables
gw_bloomington = xr.Dataset({'ar_binary_tag': gw_bloomington.ar_binary_tag, 'indices': gw_bloomington.indices})
gw_bloomington

<xarray.Dataset>
Dimensions:        (time: 109568)
Coordinates:
  * time           (time) datetime64[ns] 1980-01-01 ... 2017-06-30T21:00:00
    ens            float64 1.0
    lat            float64 39.0
    lev            float64 1e+03
    lon            float64 -86.25
Data variables:
    ar_binary_tag  (time) int8 dask.array<chunksize=(2928,), meta=np.ndarray>
    indices        (time) int64 0 1 2 3 4 ... 109563 109564 109565 109566 109567

In [5]:
""" Find times with AR conditions """

bloomington_ar = gw_bloomington.where(gw_bloomington['ar_binary_tag'] == 1, drop = True)
bloomington_ar

<xarray.Dataset>
Dimensions:        (time: 12400)
Coordinates:
  * time           (time) datetime64[ns] 1980-01-07T03:00:00 ... 2017-06-30T2...
    ens            float64 1.0
    lat            float64 39.0
    lev            float64 1e+03
    lon            float64 -86.25
Data variables:
    ar_binary_tag  (time) float64 dask.array<chunksize=(291,), meta=np.ndarray>
    indices        (time) float64 dask.array<chunksize=(291,), meta=np.ndarray>

In [6]:
""" Find only times in specific seasons """

bloomington_ar_season = {}
for season in ['DJF','MAM','JJA','SON']:
    bloomington_ar_season[season] = bloomington_ar.where(bloomington_ar['time.season'] == season, drop = True)
    # load the dataset
    #bloomington_ar_season[season].load()
bloomington_ar_season['annual'] = bloomington_ar    
bloomington_ar_season['SON']

<xarray.Dataset>
Dimensions:        (time: 3342)
Coordinates:
  * time           (time) datetime64[ns] 1980-09-01 ... 2016-11-30T15:00:00
    ens            float64 1.0
    lat            float64 39.0
    lev            float64 1e+03
    lon            float64 -86.25
Data variables:
    ar_binary_tag  (time) float64 dask.array<chunksize=(66,), meta=np.ndarray>
    indices        (time) float64 dask.array<chunksize=(66,), meta=np.ndarray>

In [7]:
""" Write the indices to disk """

for season in bloomington_ar_season:
    indices = bloomington_ar_season[season]['indices'].values
    np.savetxt(f"merra2_index_files/bloomington_ars_{season}_{algorithm}.txt",indices,fmt = "%i")

# ERA5

This section converts the MERRA2 indices to indices in ERA5, assuming that the dataset is opened from 1979-01-01 00Z at hourly resolution, with time indices increasing linearly.

In [8]:
""" Create a time axis representing ERA5 """
# create a time axis representing ERA5
e5_times = pd.DataFrame(dict(time = pd.date_range('1979-01-01 00:00:00', '2021-12-01 00:00:00', freq = '1H')))
# set the index to be the time axis
e5_times = e5_times.reindex(e5_times['time'])#.to_xarray()
# re-add the time variable (its both index and variable)
e5_times['time'] = e5_times.index
# convert to an xarray object
e5_times = e5_times.to_xarray()
# add an indices variable
e5_times['indices'] = xr.DataArray(np.arange(0,len(e5_times.time), dtype = int), dims = ('time',))
e5_times

<xarray.Dataset>
Dimensions:  (time: 376201)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-01T01:00:00 ... 2021-12-01
Data variables:
    indices  (time) int64 0 1 2 3 4 5 ... 376196 376197 376198 376199 376200

In [9]:
""" Convert to ERA5 indices and save. """
# loop over seasons
for season in bloomington_ar_season:
    # pull out the indices for the given season
    e5_indices = e5_times.sel(time = bloomington_ar_season[season].time)['indices'].values
    # save to disk
    np.savetxt(f"era5_index_files/bloomington_ars_{season}_era5_{algorithm}.txt",e5_indices,fmt = "%i")